In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import librosa.display
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.neural_network import MLPClassifier #for MLP classifier
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import * #for CNN and LSTM, * means import everything
from tensorflow.keras.optimizers import SGD,  Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import pandas as pd
import io
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.svm import SVC
import glob
import gc


In [ ]:
#data extraction from the image folder
data_path = glob.glob('../input/speechimagescolored/Image_Ruben/*.*')

def get_emotion(file_path):
    s1 = ''.join(reversed(file_path))
    emotion = ""
    for i in range(4, len(s1)):
        try:
            if(type(int(s1[i])) == int):
                return emotion
        except:
            emotion = s1[i] + emotion
            pass


data = []
labels = []

#loading of the from the filepath
for i in data_path:
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (128,128))
    image=np.array(image)
    data.append(image)
    emotion = get_emotion(i)
    labels.append(emotion)

In [ ]:
import random 
random.seed(10)

c = list(zip(data, labels))
random.shuffle(c)
data_suffle, labels_suffle = zip(*c)
X = np.array(data_suffle)
Y = np.array(labels_suffle)

#encoding and data normalization

X = X.astype('float32')
X /= 255
encoder = OneHotEncoder()


In [ ]:
height = X.shape[1]
width = X.shape[2]
color_chanels =  X.shape[3]

#costum model architecture
def creatModel():
    
    model = Sequential()
    model.add(Conv2D(256, kernel_size=(5,5), strides=1, padding='same', activation='relu', input_shape=(height, width, color_chanels)))
    model.add(MaxPooling2D(pool_size=(5,5), strides = 2, padding = 'same'))

    model.add(Conv2D(256, kernel_size=(5,5), strides=1, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(5,5), strides = 2, padding = 'same'))

    model.add(Conv2D(128, kernel_size=(5,5), strides=1, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(5,5), strides = 2, padding = 'same'))
    model.add(Dropout(0.3))

    model.add(Conv2D(64, kernel_size=(5,5), strides=1, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(5,5), strides = 2, padding = 'same'))

    model.add(Flatten())
    model.add(Dense(units=32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(units=8, activation='softmax'))
    #model.summary()
              

    return model 
              

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model

#transfer learning mode architecture
def transferModel():    
   
    vgg_model = VGG16(weights = 'imagenet',include_top=False, input_shape=(128, 128, 3))
    
    for layer in vgg_model.layers:
        layer.trainable = False
    
    x = vgg_model.output
    x = Flatten()(x) # Flatten dimensions to for use in FC layers
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x) # Dropout layer to reduce overfitting
    x = Dense(256, activation='relu')(x)
    x = Dense(8, activation='softmax')(x) # Softmax for multiclass
    transfer_model = Model(inputs=vgg_model.input, outputs=x)
    

    return transfer_model            


In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

#5-fold validation

skf = StratifiedKFold(n_splits = 5, random_state = 10, shuffle = True) 
test_accu = []
test_loss = []
kFoldMetrics = []

X_data = X
fold_var = 1

for train_index, test_index in skf.split(np.zeros(len(Y)),Y):
    
    print('------------------------------------------------------ Fold ' + str(fold_var) + ' ----------------------------------------------------')
    
    training_data = X_data[train_index]
    test_data = X_data[test_index]
    train_label = encoder.fit_transform(np.array(Y[train_index]).reshape(-1,1)).toarray()
    test_label  = encoder.fit_transform(np.array(Y[test_index]).reshape(-1,1)).toarray()

    #CREATE CNN COSTUM MODEL or TRANSFER LEARNING ONE
    model =  creatModel()
    #model =  transferModel()
    
    # COMPILE MODEL
    learning_rate= 0.00005
    batch_size = 32
    epochs = 100
    early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)
    model.compile(loss="categorical_crossentropy", optimizer= Adam(learning_rate=learning_rate), metrics = ['acc'])
    
    history = model.fit(training_data, train_label,  
                                 batch_size = batch_size, 
                                 epochs= epochs, 
                                 validation_split = 0.2, 
                                 callbacks=[early_stopping_callback])
    
    model_name = 'model_Fold_' + str(fold_var) + '.h5'
    model.save(model_name) 
    
    #Plot and save history
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1,len(acc)+1)
    plt.figure()
    plt.plot(epochs, acc, 'b', label = 'Training accuracy')
    plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.savefig('AccuracyFold' + str(fold_var) + '.jpg')
    plt.figure()
    plt.plot(epochs, loss, 'b', label = 'Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.savefig('LossFold'+str(fold_var)+'.jpg')
    

    #classification_report_fold(report,kFoldMetrics )  
    results = model.predict(test_data)    
    y_pred = encoder.inverse_transform(results)
    y_test = encoder.inverse_transform(test_label)
    report = classification_report(y_test, y_pred, output_dict=True)
    df = pd.DataFrame(report).transpose()
    kFoldMetrics.append(df)
    
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize = (12, 10))
    cm = pd.DataFrame(cm , index = [i for i in encoder.categories_] , columns = [i for i in encoder.categories_])
    sns.heatmap(cm, linecolor='white', cmap='Blues', linewidth=1, annot=True, fmt='')
    plt.title('Confusion Matrix', size=20)
    plt.xlabel('Predicted Labels', size=14)
    plt.ylabel('Actual Labels', size=14)
    plt.savefig('ConfusingMatrix'+str(fold_var)+'.jpg')

    results = dict(zip(model.metrics_names,results))
    test_accu.append(results['acc'])
    test_loss.append(results['loss'])
    
    #to clear trash and get som ram back
    tf.keras.backend.clear_session()
    gc.collect()
    fold_var += 1

    

kFoldResults_CNN_Home_Made = pd.concat([kFoldMetrics[0], kFoldMetrics[1], kFoldMetrics[2], kFoldMetrics[3], kFoldMetrics[4]])
kFoldResults_CNN_Home_Made.to_csv('5kFoldResults_CNN_Custom.csv', index=True)
#kFoldResults_CNN_Home_Made.to_csv('5kFoldResults_CNN_VGG16_transfer.csv', index=True)
